In [1]:
# +-------------------------------------------------------------------------------------------------
# Author:cfolkers
# Ministry, Division, Branch: WLRS, GEOBC, Geospatial Services 
# Created Date: 2024/05/23
# Updated Date: 
# Description: quick script to summarize burn severity in Boreal caribou herds, by herd and mgmt type
# +-------------------------------------------------------------------------------------------------


In [25]:
#import packages
import geopandas as gpd
import pandas as pd 
import feature_download
from shapely.geometry import shape

#call Feature downloader
wfs = feature_download.WFS_downloader()

In [26]:
#local variables 
pbcrp=r'/GR_2024_528/source data/incoming/BorealCaribouRecovery.gdb'
layer='PBCPR_Cleaned'
out_xlsx=r'/GR_2024_660/deliverables/Burn_Severity_By_Herd_MGMT.xlsx'

In [ ]:
#get gdfs 

herd=wfs.get_data(dataset='WHSE_WILDLIFE_INVENTORY.GCPB_CARIBOU_POPULATION_SP', query="""HERD_NUMBER IN (25,26,27,28,29)""")

fire_current=wfs.get_data(dataset='WHSE_LAND_AND_NATURAL_RESOURCE.PROT_CURRENT_FIRE_POLYS_SP', 
                               query="""FIRE_NUMBER LIKE '%G8%' AND FIRE_SIZE_HECTARES >=20 OR FIRE_NUMBER LIKE '%G9%' AND FIRE_SIZE_HECTARES >=20""")

fire_hist=wfs.get_data(dataset='WHSE_LAND_AND_NATURAL_RESOURCE.PROT_HISTORICAL_FIRE_POLYS_SP',
                            query="""FIRE_YEAR >=2016 AND FIRE_NUMBER LIKE '%G8%' AND FIRE_SIZE_HECTARES>=20 OR 
                            FIRE_YEAR >=2016 AND FIRE_NUMBER LIKE '%G9%' AND FIRE_SIZE_HECTARES>=20 """)

severity_current=wfs.get_data(dataset='WHSE_FOREST_VEGETATION.VEG_BURN_SEVERITY_SAME_YR_SP', query="""FIRE_NUMBER LIKE '%G8%' OR
    FIRE_NUMBER LIKE '%G9%'""")

severity_hist=wfs.get_data(dataset='WHSE_FOREST_VEGETATION.VEG_BURN_SEVERITY_SP',
                                query="""FIRE_YEAR >=2016 AND FIRE_NUMBER LIKE '%G8%' OR FIRE_YEAR >=2016 AND FIRE_NUMBER LIKE '%G9%' """)

mgmt_typ=gpd.read_file(filename=pbcrp,layer=layer)


In [4]:
#change all geometry columns to the same common name 
herd.rename_geometry('geo', inplace=True)
fire_current.rename_geometry('geo', inplace=True)
fire_hist.rename_geometry('geo', inplace=True)
severity_current.rename_geometry('geo', inplace=True)
severity_hist.rename_geometry('geo', inplace=True)
mgmt_typ.rename_geometry('geo', inplace=True)

In [5]:
#columns to keep
herd_cols=['HERD_NUMBER','HERD_NAME','geo']
fire_cols=['FIRE_NUMBER','FIRE_YEAR','geo']
severity_cols=['FIRE_NUMBER','FIRE_YEAR','BURN_SEVERITY_RATING','geo']
mgmt_cols=['Management','geo']
#drop columns
herd.drop(herd.columns.difference(herd_cols), axis= 1, inplace= True)
fire_current.drop(fire_current.columns.difference(fire_cols),axis =1, inplace= True)
fire_hist.drop(fire_hist.columns.difference(fire_cols),axis= 1, inplace= True)
severity_current.drop(severity_current.columns.difference(severity_cols),axis= 1, inplace= True)
severity_hist.drop(severity_hist.columns.difference(severity_cols),axis=1, inplace= True)
mgmt_typ.drop(mgmt_typ.columns.difference(mgmt_cols),axis=1, inplace= True)

In [6]:
#expolde any multipart polygons into singles and ensure crs set to 3005
herd=herd.explode(index_parts=False)
herd=herd.set_crs(3005)
fire_current=fire_current.explode(index_parts=False)
fire_current=fire_current.set_crs(3005)
fire_hist=fire_hist.explode(index_parts=False)
fire_hist=fire_hist.set_crs(3005)
severity_current=severity_current.explode(index_parts=False)
severity_current=severity_current.set_crs(3005)
severity_hist=severity_hist.explode(index_parts=False)
severity_hist=severity_hist.set_crs(3005)
mgmt_typ=mgmt_typ.explode(index_parts=False)
mgmt_typ=mgmt_typ.set_crs(3005)

In [7]:
#spatial join current and historic severity and remove any that overlap
sev_overlap=gpd.sjoin(severity_hist, severity_current,how='inner', predicate='intersects')
overlap_ind=sev_overlap.index
severity_hist_clean=severity_hist.loc[~severity_hist.index.isin(overlap_ind)]

In [ ]:
#check to see number of rows in cleaned and OG burn severity hist and plot to look for dif between current and hist 
print(f"cleaned records {len(severity_hist_clean)}")
print(f" OG Records {len(severity_hist)}")
ax=severity_current.plot(color='green')
severity_hist_clean.plot(ax=ax, color='red')

In [12]:
# spatial join (sjoin) herd and mgmt type
severity_current_sjoin1=gpd.sjoin(severity_current, herd ,how='left', predicate='intersects')
#drop index right as it can't be joined multiple times
severity_current_sjoin1 = severity_current_sjoin1.drop(columns=['index_right'])
severity_current_sjoin=gpd.sjoin(severity_current_sjoin1, mgmt_typ,how='left', predicate='intersects')


In [14]:
# spatial join (sjoin) herd and mgmt type
severity_hist_sjoin1=gpd.sjoin(severity_hist_clean, herd ,how='left', predicate='intersects')
#drop index right as it can't be joined multiple times
severity_hist_sjoin1 = severity_hist_sjoin1.drop(columns=['index_right'])
severity_hist_sjoin=gpd.sjoin(severity_hist_sjoin1, mgmt_typ,how='left', predicate='intersects')

In [ ]:
#inspect data
severity_current_sjoin

In [ ]:
#inspect
severity_hist_sjoin

In [19]:
#calculate area (km2)
severity_hist_sjoin['area']=severity_hist_sjoin.area
severity_hist_sjoin['km2']=severity_hist_sjoin['area']/1000000
severity_current_sjoin['area']=severity_current_sjoin.area
severity_current_sjoin['km2']=severity_current_sjoin['area']/1000000

In [ ]:
#filter, not needed but good to know how to do
filtered=severity_hist_sjoin[severity_hist_sjoin['HERD_NAME'].notnull()]
filtered

In [29]:
#create combined burn severity 
combined_sev=pd.concat([severity_current_sjoin,severity_hist_sjoin])

In [32]:
#group by and sum 
hist_out=severity_hist_sjoin[['BURN_SEVERITY_RATING','Management', 'HERD_NAME','km2']].groupby(['HERD_NAME','Management','BURN_SEVERITY_RATING']).sum()
curr_out=severity_current_sjoin[['BURN_SEVERITY_RATING','Management', 'HERD_NAME','km2']].groupby(['HERD_NAME','Management','BURN_SEVERITY_RATING']).sum()
combined_out=combined_sev[['BURN_SEVERITY_RATING','Management', 'HERD_NAME','km2']].groupby(['HERD_NAME','Management','BURN_SEVERITY_RATING']).sum()

In [ ]:
#export spreadsheet with multiple tabs 
with pd.ExcelWriter(out_xlsx) as writer:
    curr_out.to_excel(writer, sheet_name='Current Burn Severity, index=False')
    hist_out.to_excel(writer, sheet_name='Historic Burn Severity, index=False')
    combined_out.to_excel(writer, sheet_name='Combined Burn Severity, index=False')
    